In [15]:
import pandas as pd 
import scsims
from sklearn.preprocessing import LabelEncoder
import sys
import anndata as an
import scanpy as sp
import h5py

sys.path.append('../src')
sys.path.append('../tests')

from models.lib.lightning_train import DataModule, generate_trainer
from models.lib.data import *
from models.lib.neural import *
from pytorch_lightning.loggers import WandbLogger
from torchmetrics.functional import *
import pandas as pd
from scipy.sparse import csr_matrix
from pytorch_lightning.callbacks.early_stopping import EarlyStopping

In [17]:
module = DataModule(
    datafiles=['../data/interim/whole_brain_bhaduri_T.csv'],
    labelfiles=['allen_cortex_labels_corrected.csv'],
    class_label='subclass_label',
    index_col='index',
    batch_size=16,
    num_workers=0,
    shuffle=True,
    drop_last=True,
    normalize=True,
    skip=3,
    assume_numeric_label=False,
)

module.setup()

Creating train/val/test DataLoaders...
Done, continuing to training.
Calculating weights


In [18]:
next(iter(module.trainloader))

KeyboardInterrupt: 

In [14]:
labels = pd.read_csv('../data/raw/labels/allen_cortex_labels.tsv', sep='\t')
labels = labels[labels['subclass_label'] != 'Exclude'].reset_index()
labels.to_csv('allen_cortex_labels_corrected.csv', index=False)
pd.read_csv('allen_cortex_labels_corrected.csv')

,index,sample_name,cluster_label,class_label,subclass_label,donor_sex_label,region_label,cortical_layer_label,cell_type_accession_label,cell_type_alias_label,cell_type_alt_alias_label,cell_type_designation_label,external_donor_name_label
0,1,LS-15005h_S02_E1-50,Inh L2-5 VIP TOX2,GABAergic,VIP,M,MTG,L1,CS1910121032,Inh L2-5 VIP TOX2,NaN,Neuron 032,H200.1025
1,2,LS-15005h_S03_E1-50,Inh L1 LAMP5 GGT8P,GABAergic,LAMP5,M,MTG,L1,CS1910121002,Inh L1 LAMP5 GGT8P,NaN,Neuron 002,H200.1025
2,3,LS-15005h_S04_E1-50,Inh L1 LAMP5 NDNF,GABAergic,LAMP5,M,MTG,L1,CS1910121001,Inh L1 LAMP5 NDNF,NaN,Neuron 001,H200.1025
3,4,LS-15005h_S05_E1-50,Inh L1-3 VIP ZNF322P1,GABAergic,VIP,M,MTG,L1,CS1910121034,Inh L1-3 VIP ZNF322P1,NaN,Neuron 034,H200.1025
4,5,LS-15005h_S06_E1-50,Inh L3 VIP CBLN1,GABAergic,VIP,M,MTG,L1,CS1910121035,Inh L3 VIP CBLN1,NaN,Neuron 035,H200.1025
...,...,...,...,...,...,...,...,...,...,...,...,...,...
47504,49489,SM-GE4QU_S187_E1-50,Astro L1-6 FGFR3 ETNPPL,Non-neuronal,Astrocyte,M,A1C,WM,CS1910121111,Astro L1-6 FGFR3 ETNPPL,NaN,Non-neuron 001,H200.1025
47505,49490,SM-GE4QU_S189_E1-50,Exc L6 THEMIS LINC00343,Glutamatergic,IT,M,A1C,WM,CS1910121075,Exc L6 THEMIS LINC00343,NaN,Neuron 075,H200.1025
47506,49491,SM-GE4QU_S190_E1-50,Inh L1 LAMP5 NDNF,GABAergic,LAMP5,M,A1C,WM,CS1910121001,Inh L1 LAMP5 NDNF,NaN,Neuron 001,H200.1025
47507,49492,SM-GE4QU_S191_E1-50,Oligo L4-6 OPALIN,Non-neuronal,Oligodendrocyte,M,A1C,WM,CS1910121116,Oligo L4-6 OPALIN,NaN,Non-neuron 006,H200.1025
